<a href="https://colab.research.google.com/github/Sanazzzmi/Compare-rule-lexicon-with-lexicon-To-Sentiment-analysis-of-the-Persian-text/blob/main/Compare_rule_%26_lexicon_with_lexicon_To_Sentiment_analysis_of_the_Persian_text_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install and import data

download lexicon url
https://github.com/Text-Mining/Persian-Sentiment-Resources/blob/master/PerSent.xlsx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install hazm
!test -f resources-0.5.zip || curl -LO https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
!test -d resources || ( mkdir -p resources && cd resources && unzip ../resources-0.5.zip )

import hazm
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import random

#dataset
!wget https://raw.githubusercontent.com/ashalogic/Persian-Sentiment-Analyzer/master/Tutorial_Dataset.csv
csv_dataset = pd.read_csv("/content/Tutorial_Dataset.csv")

#lexicon
################## Saved address of the downloaded lexicon file #####################

lexicon = pd.read_excel("/content/drive/MyDrive/PerSent.xlsx")

################### pd.read_excel("Saved address of the downloaded lexicon file") ###

     |████████████████████████████████| 316 kB 5.4 MB/s 
     |████████████████████████████████| 233 kB 48.1 MB/s 
     |████████████████████████████████| 1.4 MB 46.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=bd2f63d9ee358f45f1eb85a6af95be4e4b55c56773474fdc0fa3c450dce4668a
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153943 sha256=58565ddc8cd821795a3bd20e9a3abc85bdb0c03c99c2eec23cc2a46b58f74629
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  U

### preprocessing data

In [ ]:
def CleanPersianText(text):
  _normalizer = hazm.Normalizer()
  text = _normalizer.normalize(text)
  return text

revlist = list(map(lambda x: [CleanPersianText(x[0]),x[1]],zip(csv_dataset['Text'],csv_dataset['Suggestion'])))
pos=list(filter(lambda x: x[1] == 1,revlist))
nat=list(filter(lambda x: x[1] == 2,revlist))
neg=list(filter(lambda x: x[1] == 3,revlist))

revlist_shuffle = pos[:450] + neg[:450]
random.shuffle(revlist_shuffle)

print("Posetive count {}".format(len(pos)))
print("Negetive count {}".format(len(neg)))
print("Natural  count {}".format(len(nat)))
print()
print("Total    count {}".format(len(revlist)))
print()
print("Posetive count : ","\n",pos[random.randrange(1,len(pos))])
print("Negetive count : ","\n",neg[random.randrange(1,len(neg))])
print("unknown  count : ","\n",nat[random.randrange(1,len(nat))])
print("Total    count {}".format(len(revlist_shuffle)))

Posetive count 2382
Negetive count 460
Natural  count 419

Total    count 3261

Posetive count :  
 ['همیشه عالی ', 1]
Negetive count :  
 ['دوستان در خریدش شک نکنید اندازه کوچک و قدرت بالا عالیه ', 3]
unknown  count :  
 ['این در باز کن فقط برای باز کردن درب بطری نوشیدنی کاربرد داره و واسه استفاده خانگی ضروری به نظر نمیاد بهتر بود که امکان باز کردن قوطی و کنسرو رو بهش اضافه میکردن ', 2]
Total    count 900


### just lexicon

In [ ]:
from statistics import *

lemmatizer = Lemmatizer()
tagger = POSTagger(model='resources/postagger.model')

li_key = []
li_value = []
import statistics
 
m = []
t = []
for i in range(len(revlist_shuffle[:])):
  text = revlist_shuffle[i][0]
  tag = tagger.tag(word_tokenize(text))
  lem_tag = [] 
  value_polarity = []
  for x in tag:
    lem_tag.append(lemmatizer.lemmatize(x[0]))
  idx =[]
  for lem in tag:
    if lem[0] in lexicon['Words'].tolist():
      idx.append(lexicon['Words'].tolist().index(lem[0]))
  value = []
  if len(idx) > 0:
    for x in idx:
        value.append(lexicon['Polarity'].tolist()[x])
  else:
    value.append(1)
  if statistics.mean(value) < 0:
    m.append(0)
  else:
    m.append(1)
  t.append(revlist_shuffle[i][1])

from sklearn.metrics import accuracy_score
print(accuracy_score(t, m))
print(accuracy_score(t, m, normalize=False))

0.4255555555555556
383


### rules and lexicon

In [ ]:
#article rule => https://www.sciencedirect.com/science/article/abs/pii/S0925231219313815
from statistics import *

lemmatizer = Lemmatizer()
tagger = POSTagger(model='resources/postagger.model')

li_key = []
li_value = []
keyword_list = ['اما' , 'اگرچه' , 'با اینکه', 'ولی']
keyword_list2 = ['حالیکه', 'صورتیکه']

for i in range(len(revlist_shuffle)):
  text = revlist_shuffle[i][0]
  tag = tagger.tag(word_tokenize(text))
  # rule 2  -----------------------------------------------------------------------------  rule 2 
  value_polarity = []
  if 'که' in text:
    #print(text.split('که')[0])
    target = text.split('که')[0]
    tag_target = tagger.tag(word_tokenize(target))
    #print(tag_target)
    lem_target_tag = [] 
    for x in tag_target:
      lem_target_tag.append(lemmatizer.lemmatize(x[0]))
    #print('lem_target_tag',lem_target_tag)
    for lem in lem_target_tag:
      #به ازای هر کلمه ای که در لکسیکون داریم
      df_have = lexicon.loc[lexicon['Words'] == lem]
      #print('df_have', df_have)
      #print('len df_have', len(df_have))
      if len(df_have) > 1:
        #print('>1')
        value_have = df_have["Polarity"].mean()
        
        value_polarity.append(value_have)
      elif len(df_have) == 1:
        #print('=1')
        value_have = df_have['Polarity']
        if value_have.values[0] != 0:
          value_polarity.append(value_have.values[0])
    #print(value_polarity)
    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(statistics.mean(value_polarity))
# rule 2  -----------------------------------------------------------------------------  rule 2 


# rule 7  -----------------------------------------------------------------------------  rule 7 

  if 'مخالف' in text: 
    
    li_key.append(i)
    li_value.append(-1)
# rule 7  -----------------------------------------------------------------------------  rule 7 
 
# rule 3  -----------------------------------------------------------------------------  rule 3 
  value_polarity = []
  if [key for key in keyword_list if key in text ]:
    new_keyword_list = [key for key in keyword_list if key in text ]

    value_polarity = []
    #print(new_keyword_list,CONDITION)
    for x in new_keyword_list:
      value_polarity = []
      #print(text.split('که')[0])
      target = text.split(x)[1]
      tag_target = tagger.tag(word_tokenize(target))
      #print(tag_target)
      lem_target_tag = [] 
      for x in tag_target:
        lem_target_tag.append(lemmatizer.lemmatize(x[0]))
      #print('lem_target_tag',lem_target_tag)
      for lem in lem_target_tag:
        #به ازای هر کلمه ای که در لکسیکون داریم
        df_have = lexicon.loc[lexicon['Words'] == lem]
        #print('df_have', df_have)
        #print('len df_have', len(df_have))
        if len(df_have) > 1:
          #print('>1')
          value_have = df_have["Polarity"].mean()
          
          value_polarity.append(value_have)
        elif len(df_have) == 1:
          #print('=1')
          value_have = df_have['Polarity']
          if value_have.values[0] != 0:
            value_polarity.append(value_have.values[0])

    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(statistics.mean(value_polarity))
# rule 3  -----------------------------------------------------------------------------  rule 3

# rule 4  -----------------------------------------------------------------------------  rule 4 
  value_polarity = []
  if [key for key in keyword_list2 if key in text ]:
    new_keyword_list2 = [key for key in keyword_list2 if key in text ]

    value_polarity = []
    #print(new_keyword_list2,CONDITION)
    for x in new_keyword_list2:
      value_polarity = []
      #print(text.split('که')[0])
      target = text.split(x)[1]
      tag_target = tagger.tag(word_tokenize(target))
      #print(tag_target)
      lem_target_tag = [] 
      for x in tag_target:
        lem_target_tag.append(lemmatizer.lemmatize(x[0]))
      #print('lem_target_tag',lem_target_tag)
      for lem in lem_target_tag:
        #به ازای هر کلمه ای که در لکسیکون داریم
        df_have = lexicon.loc[lexicon['Words'] == lem]
        #print('df_have', df_have)
        #print('len df_have', len(df_have))
        if len(df_have) > 1:
          #print('>1')
          value_have = df_have["Polarity"].mean()
          
          value_polarity.append(value_have)
        elif len(df_have) == 1:
          #print('=1')
          value_have = df_have['Polarity']
          if value_have.values[0] != 0:
            value_polarity.append(value_have.values[0])

    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(statistics.mean(value_polarity))
# rule 4  -----------------------------------------------------------------------------  rule 4

# rule 5  -----------------------------------------------------------------------------  rule 5 
  if [x for x in tag if 'که' in x ]:
    index_list = [tag.index(x) for x in tag if 'که' in x ]
    value_polarity = []
    idx = index_list[0]
    if len(tag)> idx+1 and'این' in tag[idx+1]:
      #print(text.split('این')[1])
      target = text.split('این')[1]
      value_polarity = []
      #print(text.split('که')[0])
      tag_target = tagger.tag(word_tokenize(target))
      #print(tag_target)
      lem_target_tag = [] 
      for x in tag_target:
        lem_target_tag.append(lemmatizer.lemmatize(x[0]))
      #print('lem_target_tag',lem_target_tag)
      for lem in lem_target_tag:
        #به ازای هر کلمه ای که در لکسیکون داریم
        df_have = lexicon.loc[lexicon['Words'] == lem]
        #print('df_have', df_have)
        #print('len df_have', len(df_have))
        if len(df_have) > 1:
          #print('>1')
          value_have = df_have["Polarity"].mean()
          
          value_polarity.append(value_have)
        elif len(df_have) == 1:
          #print('=1')
          value_have = df_have['Polarity']
          if value_have.values[0] != 0:
            value_polarity.append(value_have.values[0])
      
    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(statistics.mean(value_polarity))
  
  # rule 5  -----------------------------------------------------------------------------  rule 5

  # rule 9  -----------------------------------------------------------------------------  rule 9
  if [x for x in tag if x[1] == 'AJ' ]:
    word = [x[0] for x in tag if x[1] == 'AJ' ]
    #print(word)
    negative_word = [x for x in word if x.startswith("نا") or x.startswith("بی") or x.startswith("ضد") or x.startswith("زهر") or x.startswith("لا")]
    #print(negative_word)
    z = [x for x in negative_word if 'بیش' not in x]

    for x in z:
      if len(x) > 1:
        
        li_key.append(i)
        li_value.append(-1)
    #print(i)
    #if word[0].startswith("نا") or word[0].startswith("بی") or word[0].startswith("ضد") or word[0].startswith("زهر") or word[0].startswith("لا"):
      #if word[0] != 'بیش' and word[0] != 'بیشتر' and word[0] != 'لازم' and word[0] != 'بیشتری' and word[0] != 'لایک' :
        #print(x,lemmatizer.lemmatize(x[0]))
        #d[i] = -1
        #li_key.append(i)
        #li_value.append(-1)
  # rule 9  -----------------------------------------------------------------------------  rule 9
  


#___________________________________________________________________________________________________________________________________________#

import numpy as np

df = pd.DataFrame({'li_key':li_key , 'li_value':li_value})
df_gg = df.loc[df.groupby('li_key')['li_value'].apply(lambda x: x.abs().idxmax())]
df_gg.loc[df_gg.li_value < 0, 'li_value'] = 0
df_gg.loc[df_gg.li_value > 0, 'li_value'] = 1

true_value_rule = []

for i in df_gg['li_key'].tolist(): 
  true_value_rule.append(revlist_shuffle[i][1])

none_polarity_index = []
for i in range(len(revlist_shuffle)):    
  if i not in df_gg['li_key'].tolist():
    none_polarity_index.append(i)

true_value_non_polarity = []
for i in none_polarity_index:
  true_value_non_polarity.append(revlist_shuffle[i][1]) 

positive_value_of_non_polarity= [1] * len(none_polarity_index)


df_gg['true_value'] = true_value_rule

df_non_polarity = pd.DataFrame(
    {
      'li_key': none_polarity_index,
     'li_value': positive_value_of_non_polarity,
     'true_value' : true_value_non_polarity
    })

frames = [df_gg, df_non_polarity]
result = pd.concat(frames)
result.loc[result.true_value == 3, 'true_value'] = 0
result.loc[result.true_value == 1, 'true_value'] = 1


y_true = result['true_value'].tolist()
y_pred = result['li_value'].tolist()
print(accuracy_score(y_true, y_pred))
print(accuracy_score(y_true, y_pred, normalize=False))
     

0.5155555555555555
464
